# importing modules

In [1]:
import pandas  as pd
import requests as req
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import LabelEncoder
import re



# *****************************PROBLEM 1 – Reading the data*************************



# urls to the datasets

In [2]:
positive_baits="https://raw.githubusercontent.com/pfrcks/clickbait-detection/master/clickbait"
negative_baits="https://raw.githubusercontent.com/pfrcks/clickbait-detection/master/not-clickbait"


# A function to fetch the data from a URL

In [3]:

def my_read_function(url,label):
    response=req.get(url)
    text = response.text
    lines = text.split('\n')
    df=pd.DataFrame({'baits': lines})
    df.index.name = 'Index'
    df['Label'] = label
    #df.to_csv('baits_data.csv', index=False)
    return df


# Read the positive and negative datasets

In [4]:

positive_dataset = my_read_function(positive_baits,'clickbait')
negative_dataset = my_read_function(negative_baits,'not-clickbait')


In [5]:

#reading from  the positive_dataset/click_baits_dataset
df=pd.DataFrame(positive_dataset)
df



,baits,Label
Index,,
0,"Man repairs fence to contain dog, hilarity ens...",clickbait
1,Long-Term Marijuana Use Has One Crazy Side Eff...,clickbait
2,The water from his ear trickles into the bucke...,clickbait
3,You'll Never Guess What Nick Jonas Does in the...,clickbait
4,How Cruise Liners Fill All Their Unsold Cruise...,clickbait
...,...,...
810,OITNB's Taylor Schilling and Carrie Brownstein...,clickbait
811,Researchers have discovered the average penis ...,clickbait
812,Why it may be smart to wait to put on sunscree...,clickbait


In [6]:

#reading from  the positive_dataset/click_baits_dataset
df=pd.DataFrame(negative_dataset)
df


,baits,Label
Index,,
0,Congress Slips CISA Into a Budget Bill That's ...,not-clickbait
1,DUI Arrest Sparks Controversy,not-clickbait
2,It’s unconstitutional to ban the homeless from...,not-clickbait
3,A Government Error Just Revealed Snowden Was t...,not-clickbait
4,A toddler got meningitis. His anti-vac parents...,not-clickbait
...,...,...
1570,Loophole means ecstasy and loads of other drug...,not-clickbait
1571,Astronomers Watch a Supernova and See Reruns,not-clickbait
1572,"In Indian Rapists’ Neighborhood, Smoldering An...",not-clickbait


# Combining  the datasets

In [7]:

combined_dataset = pd.concat([positive_dataset, negative_dataset], ignore_index=True)
df=pd.DataFrame(combined_dataset)

df.to_csv("combined.csv",index=False) #

In [8]:



#reading the first 8 rows of the combined dataset
df.head(8)




,baits,Label
0,"Man repairs fence to contain dog, hilarity ens...",clickbait
1,Long-Term Marijuana Use Has One Crazy Side Eff...,clickbait
2,The water from his ear trickles into the bucke...,clickbait
3,You'll Never Guess What Nick Jonas Does in the...,clickbait
4,How Cruise Liners Fill All Their Unsold Cruise...,clickbait
5,Could Queen Elizabeth Veto Brexit?,clickbait
6,This Is the Worst Color to Paint Your Kitchen,clickbait
7,The Shocking Truth About Sugar,clickbait


# Shuffling  the combined dataset using numpy

In [9]:



# Create an array of indices from 0 to the length of the combined dataset
shuffled_indices = np.arange(len(combined_dataset))
# Shuffle the array of indices randomly using numpy's random.shuffle function
np.random.shuffle(shuffled_indices)
# Use the shuffled indices to rearrange the rows of the combined dataset, creating the shuffled dataset
shuffled_dataset = combined_dataset.iloc[shuffled_indices]
df2=pd.DataFrame(shuffled_dataset)
df2.to_csv("shuffled.csv",index=False)




In [10]:


#reading the first ten rows of the shuffled_dataset
df=pd.DataFrame(shuffled_dataset)
df.head(10)



,baits,Label
769,And the winning #Powerball numbers are...,clickbait
2114,Chuck Blazer: Fifa imposes life ban from footb...,not-clickbait
572,What if Apple Made iPhones in the US? Here’s H...,clickbait
2003,North Korea announces it conducted nuclear test,not-clickbait
1000,Boaty McBoatface wins poll to name polar resea...,not-clickbait
955,"One Day Later, Anonymous Already Takes Down 3,...",not-clickbait
1257,Teen Who Worked In Corner Shop For 10 Weeks To...,not-clickbait
1187,It is rare for a new animal species to emerge ...,not-clickbait
412,You Won’t Believe What 11-Time Olympic Medalis...,clickbait
2088,Turkmenistan president outlaws all sale of tob...,not-clickbait


# Split the shuffled dataset into train, validation, and test sets

In [11]:


train_data_percentage = 0.72  # 72% for training
validation_data_percentage = 0.08  # 8% for validation
test_data_percentage = 0.20  # 20% for testing



In [12]:


# Calculate the number of samples for each split
total_samples = len(shuffled_dataset) #number of rows in the combined dataset
train_data_samples = int(train_data_percentage * total_samples) #number of rows in the training dataset 
validation_data_samples = int(validation_data_percentage * total_samples) #number of rows in the validation dataset
test_data_samples = total_samples - train_data_samples - validation_data_samples #number of rows in the testing dataset



In [13]:


#printing the number of rows/ samples in each dataset
print("********************* OUTPUT ********************\n")
print(f"total sample or the number of rows ={total_samples} samples/rows")
print(f"validation samples or the number of rows ={validation_data_samples} samples/rows")
print(f"Training samples or the number of rows ={train_data_samples} samples/rows")
print(f"test samples or the number of rows ={test_data_samples} samples/rows")



********************* OUTPUT ********************

total sample or the number of rows =2390 samples/rows
validation samples or the number of rows =191 samples/rows
Training samples or the number of rows =1720 samples/rows
test samples or the number of rows =479 samples/rows


In [14]:


# Spliting the dataset into training and the remaining data (remaining_data)
training_data, remaining_data = train_test_split(shuffled_dataset, test_size=(1 - train_data_percentage))

# Split the remaining data into validation and test sets
validation_data, test_data = train_test_split(remaining_data, test_size=test_data_percentage / (test_data_percentage + validation_data_percentage))
#saving the datasets 
train_dataset=pd.DataFrame(training_data) #training dataset
train_dataset.to_csv("traning_data.csv",index=False)

validating_dataset=pd.DataFrame(validation_data) #validating  set
validating_dataset.to_csv("validating_data.csv",index=False)

testing_dataset=pd.DataFrame(test_data) #testing set 
testing_dataset.to_csv('testing_dataset.csv',index=False)




In [15]:


#reading from each and every dataset after split
train_dataset #reading from the training dataset



,baits,Label
746,Oscars 2016: And the winners are...,clickbait
474,You’ll Be Shocked at How Little An Oscar Statu...,clickbait
1114,Brussels attacks: Zaventem and Maelbeek bombs ...,not-clickbait
1634,Saudi court sentences poet to death for renoun...,not-clickbait
1059,Mayor of Santa Ana Named in Pot Shop Raid Lawsuit,not-clickbait
...,...,...
2021,Greeks defy Europe with overwhelming referendu...,not-clickbait
283,‘Game Of Thrones’: How Will Brexit Impact Fund...,clickbait
149,"Enormous ""Frog Penis"" That Went Viral Isn't A ...",clickbait
32,Man swallowed a microSD card and you won't bel...,clickbait


In [16]:


#reading from the testing dataset
testing_dataset



,baits,Label
248,Crowdfunding for Kanye’s debt raises a meager...,clickbait
977,New York City sets new record: no murders for ...,not-clickbait
925,Lab tech allegedly faked result in drug case; ...,not-clickbait
2097,Oil price falls below $35 a barrel to fresh 11...,not-clickbait
1028,Chicago police unions fight to destroy decades...,not-clickbait
...,...,...
316,Why Do Cats Come To The Bathroom With You?,clickbait
2159,Japan furious at UNESCO listing Nanjing Massac...,not-clickbait
213,Why obsessed hockey fans are updating ‘NHL 200...,clickbait
1222,Officers in Vineland fatal arrest named by 'An...,not-clickbait


In [17]:


#reading from the validation  dataset
validating_dataset



,baits,Label
580,Anthony Bourdain Reveals His Favorite City for...,clickbait
933,Student suspended after carrying classmate hav...,not-clickbait
1692,China bans depictions of gay people on television,not-clickbait
572,What if Apple Made iPhones in the US? Here’s H...,clickbait
517,You'll Never Believe How Queen Elizabeth's Cor...,clickbait
...,...,...
25,Jennifer Garner Dating Her Accountant?,clickbait
286,Details about 1960 Volkswagen Beetle - Classic,clickbait
1422,Planned Parenthood sues anti-abortion group be...,not-clickbait
1505,Apple says the FBI is making access demands ev...,not-clickbait


# Calculating the "target rate" for each dataset (training,validation and test)

In [18]:


train_data_target_rate = (train_dataset['Label'] == 'clickbait').mean()
validation_data_target_rate = (validating_dataset['Label'] == 'clickbait').mean()
test_data_target_rate = (testing_dataset['Label'] == 'clickbait').mean()



# what % of the three datasets is t is labeled as clickbait?

In [19]:
print("********************* OUTPUT ********************\n")
print(f"{train_data_target_rate.round(4)*100}% of the training data is labeled as clickbait")
print(f"{validation_data_target_rate.round(4)*100}% of the validating data is labeled as clickbait")
print(f"{test_data_target_rate.round(4)*100}% of the testing data is labeled as clickbait")



********************* OUTPUT ********************

33.26% of the training data is labeled as clickbait
36.65% of the validating data is labeled as clickbait
36.120000000000005% of the testing data is labeled as clickbait


# **********************  PROBLEM 3 – Training a single Bag-of-Words (BOW) Text Classifier ***************

In [20]:



# Loading the training and validation datasets
validating_dataset  #the dataset containing the validating data
train_dataset #the dataset containing the traingin data


# Map labels to binary values (1 for clickbait, 0 for non-clickbait)
train_dataset['Label'] = (train_dataset['Label'] == 'clickbait').astype(int)
validating_dataset['Label'] = (validating_dataset['Label'] == 'clickbait').astype(int)



# Creating  a Pipeline with CountVectorizer and MultinomialNB
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(ngram_range=(1, 2))),
    ('classifier', MultinomialNB())
])


# Fiting  the classifier on the training dataset
pipeline.fit(train_dataset['baits'], train_dataset['Label'])


# Predict on the training and validation datasets
train_predictions = pipeline.predict(train_dataset['baits'])
validation_predictions = pipeline.predict(validating_dataset['baits'])



# Computing  precision, recall, and F1-score for training and validation datasets with zero_division='warn'
train_precision = precision_score(train_dataset['Label'], train_predictions, pos_label=1, zero_division='warn')
train_recall = recall_score(train_dataset['Label'], train_predictions, pos_label=1, zero_division='warn')
train_f1 = f1_score(train_dataset['Label'], train_predictions, pos_label=1, zero_division='warn')

validation_precision = precision_score(validating_dataset['Label'], validation_predictions, pos_label=1, zero_division='warn')
validation_recall = recall_score(validating_dataset['Label'], validation_predictions, pos_label=1, zero_division='warn')
validation_f1 = f1_score(validating_dataset['Label'], validation_predictions, pos_label=1, zero_division='warn')





In [21]:


# Print the results
print("********************* OUTPUT ********************\n")
print(f"Training Precision is  {train_precision}  or  {train_precision.round(4)*100}%")
print(f"Training Recall is {train_recall}  or {train_recall.round(4)*100}%")
print(f"Training F1-score is  {train_f1} or {train_recall.round(4)*100}%")
print("\n")
print(f"Validation Precision is {validation_precision} or {validation_precision.round(4)*100}% ")
print(f"Validation Recall is  {validation_recall} or {validation_precision.round(4)*100}%")
print(f"Validation F1-score is { validation_f1} or {validation_f1.round(4)*100}%")



********************* OUTPUT ********************

Training Precision is  0.9930434782608696  or  99.3%
Training Recall is 0.9982517482517482  or 99.83%
Training F1-score is  0.995640802092415 or 99.83%


Validation Precision is 0.9264705882352942 or 92.65% 
Validation Recall is  0.9 or 92.65%
Validation F1-score is 0.9130434782608695 or 91.3%


# ********************** PROBLEM 4 – Hyperparameter Tuning**********************




In [22]:






# Mapping  labels to binary values (1 for clickbait, 0 for non-clickbait)
train_dataset['Label'] = (train_dataset['Label'] == 'clickbait').astype(int)
validating_dataset['Label'] = (validating_dataset['Label'] == 'clickbait').astype(int)


# Defining a grid of hyperparameters to search
parameter_grid = {
    'vectorizer__max_df': [0.5, 0.75, 1.0],  # Vary max_df for CountVectorizer
    'classifier__alpha': [1.0, 0.5, 0.1],     # Vary smoothing for MultinomialNB
    'vectorizer__ngram_range': [(1, 1), (1, 2)]  # Include or exclude bigrams in CountVectorizer
}


# Initializing  an empty  list to store the results
results = []


# Iterating  over the parameter grid
for params in ParameterGrid(parameter_grid):
    # Creating  a Pipeline with CountVectorizer and MultinomialNB with the current parameters
    pipeline = Pipeline([
        ('vectorizer', CountVectorizer(max_df=params['vectorizer__max_df'], ngram_range=params['vectorizer__ngram_range'])),
        ('classifier', MultinomialNB(alpha=params['classifier__alpha']))
    ])
    

    # Fiting  the classifier on the training dataset
    pipeline.fit(train_dataset['baits'], train_dataset['Label'])

    
    # Predicting  on the validation dataset
    validation_predictions = pipeline.predict(validating_dataset['baits'])

    
    # Computing  precision, recall, and F1-score for validation dataset
    validation_precision = precision_score(validating_dataset['Label'], validation_predictions,zero_division=1)
    validation_recall = recall_score(validating_dataset['Label'], validation_predictions,zero_division=1)
    validation_f1 = f1_score(validating_dataset['Label'], validation_predictions,zero_division=1)

    
    # Storing  the results
    results.append({
        'params': params,
        'validation_precision': validation_precision,
        'validation_recall': validation_recall,
        'validation_f1': validation_f1
    })

# Converting  results to a DataFrame for analysis
results_dataframe = pd.DataFrame(results)

# Sorting  the results by F1-score in descending order
results_dataframe = results_dataframe.sort_values(by='validation_f1', ascending=False)

# Displaying  the top and bottom results 
print("********************* OUTPUT ********************\n")
print("Top Results")
print(results_dataframe.head())
print("Bottom Results")
print(results_dataframe.tail())






********************* OUTPUT ********************

Top Results
                                               params  validation_precision  \
0   {'classifier__alpha': 1.0, 'vectorizer__max_df...                   1.0   
1   {'classifier__alpha': 1.0, 'vectorizer__max_df...                   1.0   
16  {'classifier__alpha': 0.1, 'vectorizer__max_df...                   1.0   
15  {'classifier__alpha': 0.1, 'vectorizer__max_df...                   1.0   
14  {'classifier__alpha': 0.1, 'vectorizer__max_df...                   1.0   

    validation_recall  validation_f1  
0                 1.0            1.0  
1                 1.0            1.0  
16                1.0            1.0  
15                1.0            1.0  
14                1.0            1.0  
Bottom Results
                                               params  validation_precision  \
5   {'classifier__alpha': 1.0, 'vectorizer__max_df...                   1.0   
4   {'classifier__alpha': 1.0, 'vectorizer__max_df...  

# *********************************PROBLEM 5 – Model selection ************************




In [23]:


#To select the best model from the results of the hyperparameter tuning,
#i choose the one that achieved the highest F1-score on the validation set. how do choose.


# Finding  the best model based on validation F1-score values
best_model = max(results, key=lambda x: x['validation_f1'])

# Displaying  the best model's parameters and validation F1-score  value
print("********************* OUTPUT ********************\n")
print("Best Model - Parameters ")
print(best_model['params'])
print("\n")
print(" Validation F1-Score value  of  the Best Model is ", best_model['validation_f1'])



********************* OUTPUT ********************

Best Model - Parameters 
{'classifier__alpha': 1.0, 'vectorizer__max_df': 0.5, 'vectorizer__ngram_range': (1, 1)}


 Validation F1-Score value  of  the Best Model is  1.0


In [24]:
 
    
#applying the  model to my  test set and computing the  precision, recall, and F1-score values
    
   # Creating  a pipeline with the best model's parameters
best_model_pipeline = Pipeline([
    ('vectorizer', CountVectorizer(max_df=best_model['params']['vectorizer__max_df'], ngram_range=best_model['params']['vectorizer__ngram_range'])),
    ('classifier', MultinomialNB(alpha=best_model['params']['classifier__alpha']))
])

# Fiting my  best model on the training dataset
best_model_pipeline.fit(train_dataset['baits'], train_dataset['Label'])

# Predicting  on the test dataset
test_predictions = best_model_pipeline.predict(testing_dataset['baits'])

#removing posible NAN values
clean_testing_data = testing_dataset.dropna()

# Convert true labels to numeric format
clean_testing_data['Label'] = clean_testing_data['Label'].map({'clickbait': 1, 'not-clickbait': 0})

# Computing the  precision, recall, and F1-score values  for the test dataset
test_precision = precision_score(clean_testing_data['Label'], test_predictions,zero_division=1)
test_recall = recall_score(clean_testing_data['Label'], test_predictions)
test_f1 = f1_score(clean_testing_data['Label'], test_predictions)




In [25]:


print("********************* OUTPUT ********************\n")
# Display the test set results
print("These are Test Set Metrics for Best Model ")
print("Precision ", test_precision)
print("Recall ", test_recall)
print("F1-Score " , test_f1)
 
    
    

********************* OUTPUT ********************

These are Test Set Metrics for Best Model 
Precision  1.0
Recall  0.0
F1-Score  0.0


# ************************PROBLEM 6 – Key Indicators*********************

In [26]:

#Convert the text to lowercase to ensure consistency.
train_dataset['baits'] = train_dataset['baits'].str.lower()



# Defining  a dictionary to map labels to binary values
label_mapping = {'clickbait': 1, 'not-clickbait': 0} # Mapping  labels to binary values (1 for clickbait, 0 for non-clickbait)



# Using  the map method to update my  'Label' column
train_dataset['Label'] = train_dataset['Label'].map(label_mapping)



# Creating  a CountVectorizer to extract key words
vectorizer = CountVectorizer(ngram_range=(1, 2)) 


# Transforming  the training data into a feature matrix
X_train = vectorizer.fit_transform(train_dataset['baits'])


# Initializing  and training  a Multinomial Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, train_dataset['Label'])


# Geting  the feature log probabilities for the "clickbait" class
clickbait_log_probs = clf.feature_log_prob_[1]


# Geting the corresponding vocabulary words
feature_names = np.array(vectorizer.get_feature_names_out())


# Creating  a DataFrame to analyze the results
clickbait_dataframe = pd.DataFrame({'Word': feature_names, 'Log Probability': clickbait_log_probs})


# Sorting  the DataFrame by log probability in descending order
clickbait_dataframe = clickbait_dataframe.sort_values(by='Log Probability', ascending=False)

ValueError: Input y contains NaN.

In [27]:






# Display the top 5 words with the highest log probability
top_clickbait_words = clickbait_dataframe.head(6)
print("********************* OUTPUT ********************\n")
print("Top 5 Clickbait Indicator Words")
print(top_clickbait_words)








NameError: name 'clickbait_dataframe' is not defined

# ************** PROBLEM 7 – Regular expression *************************************

In [28]:
# A list of words that are recognized as clickbaits in the previous Question
top_keywords = top_clickbait_words['Word'].unique()

# Constructing  a regular expression pattern to match any of the top keywords with word boundaries
pattern = r'\b(?:' + '|'.join(re.escape(keyword) for keyword in top_keywords) + r')\b'

#A testing message
text = "you will be a billionaire within two days if you do the following. Follow this steps ."
print("********************* OUTPUT ********************\n")
# Using  re.search function to find matches in the text
if re.search(pattern, text):
    print("Keyword found in the text.")
else:
    print("Keyword not found in the text.")
    



NameError: name 'top_clickbait_words' is not defined

In [29]:
#removing any nans

testing_dataset=testing_dataset.dropna()



# Defining  a dictionary to map labels to binary values
label_mapping = {'clickbait': 1, 'not-clickbait': 0} # Mapping  labels to binary values (1 for clickbait, 0 for non-clickbait)


# Using  the map method to update my  'Label' column
testing_dataset['Label'] = testing_dataset['Label'].map(label_mapping)



# Creating a function  to check if any top keywords are present in the text from the test data_set
def keyword_classifier(text):
    pattern = r'\b(?:' + '|'.join(re.escape(keyword) for keyword in top_keywords) + r')\b'
    return bool(re.search(pattern, text))




# Applying  the function to  test_dataset 

testing_dataset['Predicted'] = testing_dataset['baits'].apply(keyword_classifier)



# Calculating the  precision and recall values 

precision = precision_score(testing_dataset['Label'], testing_dataset['Predicted'], zero_division='warn')

recall = recall_score(testing_dataset['Label'], testing_dataset['Predicted'], zero_division='warn')

f1=f1_score(testing_dataset['Label'], testing_dataset['Predicted'], zero_division='warn')





NameError: name 'top_keywords' is not defined

In [287]:
#printing the results 
print("********************* OUTPUT ********************\n")
print(f"The Precision of this classifier is {precision} or  {precision.round(4)*100}%")
print(f"The Recall of this classifier is  {recall} or  {recall.round(4)*100}%")
print(f"The Recall of this classifier is  {f1} or  {f1.round(4)*100}%")


********************* OUTPUT ********************

The Precision of this classifier is 0.32589285714285715 or  32.59%
The Recall of this classifier is  0.46496815286624205 or  46.5%
The Recall of this classifier is  0.3832020997375329 or  38.32%
